In [1]:
!pwd

/home/mhossai5/Ag-Ab-Affinity2/LLM+Graph


In [2]:
import os
import re
import torch
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

# ProtT5 imports
from transformers import T5Tokenizer, T5EncoderModel
# AbLang2
import ablang2

tqdm.pandas()

2025-11-30 12:04:23.361383: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-30 12:04:23.390976: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-30 12:04:23.391006: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-30 12:04:23.392195: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-30 12:04:23.400002: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructio

[2025-11-30 12:04:26,908] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/mhossai5/.conda/envs/llm_tor2/compiler_compat/ld: warning: libstdc++.so.6, needed by /home/mhossai5/local/cuda-11.8/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/mhossai5/.conda/envs/llm_tor2/compiler_compat/ld: /home/mhossai5/local/cuda-11.8/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/home/mhossai5/.conda/envs/llm_tor2/compiler_compat/ld: /home/mhossai5/local/cuda-11.8/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/home/mhossai5/.conda/envs/llm_tor2/compiler_compat/ld: /home/mhossai5/local/cuda-11.8/lib64/libcufile.so: undefined reference to `std::ostream::tellp()@GLIBCXX_3.4'
/home/mhossai5/.conda/envs/llm_tor2/compiler_compat/ld: /home/mhossai5/local/cuda-11.8/lib64/libcufile.so: undefined reference to `std::string::substr(unsigned long, unsigned long) const@GLIBCXX_3.4'
/home/mhossai5/.conda/envs/llm_tor2/compiler_compat/ld: /home/mhossai5/local/cuda-11.8/lib64/libcufile.

In [3]:
# ============================================================
# DEVICE SETUP
# ============================================================

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"\n[INFO] Using device: {device}\n")


[INFO] Using device: cuda:0



In [4]:
# ============================================================
# LOAD ABLANG2 (Antibody Model)
# ============================================================

def load_ablang2():
    model_name = "ablang2-paired"
    print(f"[INFO] Loading AbLang2 model: {model_name}")

    model = ablang2.pretrained(
        model_to_use=model_name,
        random_init=False,
        device=device
    )
    return model

ab_model = load_ablang2()

# ============================================================
# LOAD Encoder (Antigen Model)
# ============================================================

def load_ag():
    model_name = "Rostlab/prot_t5_xl_half_uniref50-enc"
    print(f"[INFO] Loading ProtT5 model: {model_name}")

    tokenizer = T5Tokenizer.from_pretrained(
        model_name,
        do_lower_case=False
    )
    model = T5EncoderModel.from_pretrained(model_name)

    # FP16 for GPU, FP32 for CPU
    if device.type == "cuda":
        model = model.half()
    else:
        model = model.float()

    model = model.to(device).eval()
    return model, tokenizer

ag_model, ag_tokenizer = load_ag()

[INFO] Loading AbLang2 model: ablang2-paired


/home/mhossai5/.conda/envs/llm_tor2/lib/python3.10/site-packages/ablang2/load_model.py:112: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(
You are using the defau

[INFO] Loading ProtT5 model: Rostlab/prot_t5_xl_half_uniref50-enc


In [5]:
# ============================================================
# DATA LOADING & CLEANING
# ============================================================

df = pd.read_csv("https://raw.githubusercontent.com/Fengithub/symLMF-PPI/refs/heads/master/datasets/S.cerevisiae-benchmark/pros_AB.txt",sep ='\t')

print(f"[INFO] Final dataset size: {df.shape[0]} entries")
df

[INFO] Final dataset size: 11188 entries


,Protein_A_id,Protein_B_id,Protein_A_sequence,Protein_B_sequence,Protein_A_idx,Protein_B_idx,Interaction
0,P16649,P14922,MTASVSNTQNKLNELLDAIRQEFLQVSQEANTYRLQNQKDYDFKMN...,MNPGGEQTIMEQPAQQQQQQQQQQQQQQQQAAVPQQPLDPLTQSTA...,0,577,1
1,P07269,P22035,MMEEFSYDHDFNTHFATDLDYLQHDQQQQQQQQHDQQHNQQQQPQP...,MSNISTKDIRKSKPKRGSGFDLLEVTESLGYQTHRKNGRNSWSKDD...,1,1598,1
2,P33418,P50278,MLERIQQLVNAVNDPRSDVATKRQAIELLNGIKSSENALEIFISLV...,MTTTVPKVFAFHEFAGVAEAVADHVIHAQNSALKKGKVSRSTQMSG...,2,1599,1
3,P27705,P14922,MQSPYPMTQVSNVDDGSLLKESKSKSKVAAKSEAPRPHACPICHRA...,MNPGGEQTIMEQPAQQQQQQQQQQQQQQQQAAVPQQPLDPLTQSTA...,3,577,1
4,P05453,P05453,MSDSNQGNNQQNYQQYSQNGNQQQGNNRYQGYQAYNAQAQPAGGYY...,MSDSNQGNNQQNYQQYSQNGNQQQGNNRYQGYQAYNAQAQPAGGYY...,4,4,1
...,...,...,...,...,...,...,...
11183,P28241,P04147,MLRNTFFRNTSRRFLATVKQPSIGRYTGKPNPSTGKYTVSFIEGDG...,MADITDKTAEQLENLNIQDDQKQAATGSESQSVENSSASLYVGDLE...,757,614,0
11184,P23641,Q08962,MSVSAAPAIPQYSVSDYMKFALAGAIGCGSTHSSMVPIDVVKTRIQ...,MRQLTEEETKVVFEKLAGYIGRNISFLVDNKELPHVFRLQKDRVYY...,1345,1112,0
11185,P12695,P39014,MSAFVRVVPRISRSSVLTRSLRLQLRCYASYPEHTIIGMPALSPTM...,MRRERQRMMSFEDKDKDDLDNSNSNNSSEMTDTAMMPPLKRLLITG...,2353,212,0
11186,P16451,Q03081,MLSAISKVSTLKSCTRYLTKCNYHASAKLLAVKTFSMPAMSPTMEK...,MKLAQDMNVDEIFLKQAAEAIAVISSSPTHTDPIIRELLHRIRQSS...,1360,2209,0


In [6]:
# ============================================================
# EMBEDDING FUNCTIONS
# ============================================================

def embed_antibody(hseq: str, lseq: str):
    """
    Returns AbLang2 embeddings for (heavy, light) sequences.
    """
    seq = f"{hseq}|{lseq}".upper() # VH first, VL second, with | used to separated the two sequences 
    tokenized_seq = ab_model.tokenizer([seq], pad=True, w_extra_tkns=False, device=device)

    # Generate rescodings
    with torch.no_grad():
        rescoding = ab_model.AbRep(tokenized_seq).last_hidden_states
    return rescoding.squeeze().detach().cpu().numpy()#.reshape(-1)


# def embed_antigen(seq: str):
#     """
#     Returns Encoder embeddings for antigen. shape 2560
#     """
#     tokens = ag_tokenizer(
#         seq,
#         return_tensors="pt",
#         add_special_tokens=True
#     )

#     tokens = {k: v.to(device) for k, v in tokens.items()}

#     with torch.no_grad():
#         output = ag_model(**tokens)

#     return output.last_hidden_state.squeeze(0).cpu().numpy() #.reshape(-1)
def embed_antigen(seq: str):
    seq = " ".join(list(re.sub(r"[UZOB]", "X", seq)))
    ids = ag_tokenizer.encode_plus(seq, add_special_tokens=True, padding="longest")
    input_ids = torch.tensor(ids['input_ids']).unsqueeze(0).to(device)
    attention_mask = torch.tensor(ids['attention_mask']).unsqueeze(0).to(device)
    with torch.no_grad():
        embedding = ag_model(input_ids=input_ids, attention_mask=attention_mask)
    token_embeddings = embedding.last_hidden_state.squeeze(0) 
    mask = attention_mask.squeeze(0).bool()  
    filtered_embeddings = token_embeddings[mask]
    return filtered_embeddings.squeeze().detach().cpu().numpy()

In [7]:
import h5py
import numpy as np
from tqdm import tqdm
file_path = "../DeepInterAware/data/Yeast/Yeast-ProtT5-Full.h5"
!mkdir -p {os.path.dirname(file_path)}
seq_list = pd.unique(df[['Protein_A_sequence','Protein_B_sequence']].values.ravel())
with h5py.File(file_path, 'w') as hf:
    for seq in tqdm(seq_list):
        emb = embed_antigen(seq).reshape(-1)
        hf.create_dataset(seq, data=emb, compression="gzip")
print(f"\n[INFO] Embeddings will be saved to:\n{file_path}\n")

100%|██████████| 2526/2526 [02:52<00:00, 14.64it/s]



[INFO] Embeddings will be saved to:
../DeepInterAware/data/Yeast/Yeast-ProtT5-Full.h5



In [8]:
df.iloc[:1].progress_apply(lambda x: embed_antigen(x['Protein_A_sequence']).shape, axis= 1)

  0%|          | 0/1 [00:00<?, ?it/s]

0    (714, 1024)
dtype: object

In [9]:
df

,Protein_A_id,Protein_B_id,Protein_A_sequence,Protein_B_sequence,Protein_A_idx,Protein_B_idx,Interaction
0,P16649,P14922,MTASVSNTQNKLNELLDAIRQEFLQVSQEANTYRLQNQKDYDFKMN...,MNPGGEQTIMEQPAQQQQQQQQQQQQQQQQAAVPQQPLDPLTQSTA...,0,577,1
1,P07269,P22035,MMEEFSYDHDFNTHFATDLDYLQHDQQQQQQQQHDQQHNQQQQPQP...,MSNISTKDIRKSKPKRGSGFDLLEVTESLGYQTHRKNGRNSWSKDD...,1,1598,1
2,P33418,P50278,MLERIQQLVNAVNDPRSDVATKRQAIELLNGIKSSENALEIFISLV...,MTTTVPKVFAFHEFAGVAEAVADHVIHAQNSALKKGKVSRSTQMSG...,2,1599,1
3,P27705,P14922,MQSPYPMTQVSNVDDGSLLKESKSKSKVAAKSEAPRPHACPICHRA...,MNPGGEQTIMEQPAQQQQQQQQQQQQQQQQAAVPQQPLDPLTQSTA...,3,577,1
4,P05453,P05453,MSDSNQGNNQQNYQQYSQNGNQQQGNNRYQGYQAYNAQAQPAGGYY...,MSDSNQGNNQQNYQQYSQNGNQQQGNNRYQGYQAYNAQAQPAGGYY...,4,4,1
...,...,...,...,...,...,...,...
11183,P28241,P04147,MLRNTFFRNTSRRFLATVKQPSIGRYTGKPNPSTGKYTVSFIEGDG...,MADITDKTAEQLENLNIQDDQKQAATGSESQSVENSSASLYVGDLE...,757,614,0
11184,P23641,Q08962,MSVSAAPAIPQYSVSDYMKFALAGAIGCGSTHSSMVPIDVVKTRIQ...,MRQLTEEETKVVFEKLAGYIGRNISFLVDNKELPHVFRLQKDRVYY...,1345,1112,0
11185,P12695,P39014,MSAFVRVVPRISRSSVLTRSLRLQLRCYASYPEHTIIGMPALSPTM...,MRRERQRMMSFEDKDKDDLDNSNSNNSSEMTDTAMMPPLKRLLITG...,2353,212,0
11186,P16451,Q03081,MLSAISKVSTLKSCTRYLTKCNYHASAKLLAVKTFSMPAMSPTMEK...,MKLAQDMNVDEIFLKQAAEAIAVISSSPTHTDPIIRELLHRIRQSS...,1360,2209,0
